# Machine Learning Math 1

This series starts with introducing you to the math that allows you to *interpret* a linear regression.

## Machine Learning Intuition

### Plain English

In a lot of computer science, we understand algorithms by assuming an oracle machine where you feed it with inputs and it produces the correct output every time. Essentially, black magic hiding inside of black boxes.

* [Oracle Machine](https://en.wikipedia.org/wiki/Oracle_machine)

Can we create something that isn't a black box that will produce the correct output every time? We can't be sure. Maybe no oracle machine can be built. Maybe building such an oracle machine is extremely complex and time-consuming.

What if we could step back and ask the question, "What if an almost-oracle is good enough?"

At a very abstract level, if we have many data points, we can theoretically summarize the data points with a model that says something meaningful or illuminating about the data. While it isn't necessarily 100% correct, this model can be used to understand both the data points that we've seen as well as data points we've not yet seen.

* [Obligatory George Box quote](https://en.wikipedia.org/wiki/All_models_are_wrong)

The process of creating these almost-oracles based on data points is known as machine learning.

In machine learning, you choose some class of models you have reason to believe will provide you with almost correct outputs given what you understand about the inputs. You feed your model with the input data, you provide some hints, and then you ask the model to train itself. By observing the outputs it creates, the model adjusts itself to provide increasingly correct answers as outputs.

To give you a better sense of what this means in practice, we'll talk about the model you created in the machine learning workshop: linear regression.

### Matrix Transformation

Let's start by pretending that you want to describe an object, say a house. To create this description, you will want to capture many aspects of the house. You might record values like the size of the house, the size of the backyard, the number of bedrooms and bathrooms in the house. Each of these values that you choose to collect is referred to as a *feature*.

Assume you have $m$ different objects that you've described with $n$ features. You can label the specific feature of a specific object as $x_{i,j}$, where $i$ corresponds to the ID of the object, and $j$ corresponds to the index of the feature. This allows us to represent these $m$ data points as an $m \times n$ matrix.

$$
\begin{bmatrix}
x_{1,1} & \cdots & x_{1,n} \\
x_{2,1} & \cdots & x_{2,n} \\
\vdots & \ddots & \vdots \\
x_{m,1} & \cdots & x_{m,n}
\end{bmatrix}
$$

Let's assume that there is an additional feature that is intuitively related to the features we collected. In our case, it might be the price of the house in question. Because we have $m$ different values for this additional feature, we can represent these as an $m \times 1$ column vector.

$$
\begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_m
\end{bmatrix}
$$

So to repeat what we just said about the almost-oracles in a more mathematical way, machine learning is the process of deriving transformations of the input matrix into the output matrix.

## Define the Task

Now that you know what machine learning is, the next step is to recognize that machine learning is a tool that serves to answer important questions. Therefore, the first step in machine learning is to decide what question you wish to answer.

* [Machine Learning Yearning](https://gallery.mailchimp.com/dc3a7ef4d750c0abfc19202a3/files/Machine_Learning_Yearning_V0.5_01.pdf)

To help you understand what these questions might look like in the real world, let's do some roleplay.

Something that happens occasionally in dramas is there will be an evil corporation that is looking to buy out neighborhoods in order to construct resorts or hotels that will pull in tourists and make lots of money for the corporation.

* [Princess Jellyfish](https://myanimelist.net/anime/8129/Kuragehime)

Imagine that you are an executive at one such evil hotel corporation, Hotelray.

You'd like to find places for your hotels that will make you a lot of money (surrounded by high value homes or tourist attractions) but wouldn't cost you very much money in terms of construction (low cost of labor, relatively inexpensive to buy out).

## Operationalize the Task

The next step is to identify what it is that you wish to predict in order to answer your question and identify data that you can bring together that might help in this prediction. This is known as *operationalizing the question*.

* [Operationalization](https://en.wikipedia.org/wiki/Operationalization)

In our case, we can answer a substantial amount of Hotelray's business question by simply finding out what the prices of the houses are. Unfortunately, not every house will publish their market value, and sending out an appraiser to every house of every possible neighborhood would be extremely expensive and most likely unwelcomed by the current residents.

However, an "almost-oracle" that had reasonable guesses about the price of every home would be good enough.

We're in luck! It turns out that we have data from the [House Sales in King County, USA](https://www.kaggle.com/harlfoxem/housesalesprediction), which is available from Kaggle data sets. If we can create a model that had reasonable guesses about the price of every home in that data set, it might also have reasonable guesses about every house in every neighborhood we're interested in.

I've taken advantage of the [CC0: Public Domain License](https://creativecommons.org/publicdomain/zero/1.0/) which permits copying the data set for any purpose, and I've pushed it to the static hosting service of [WeDeploy](https://www.wedeploy.com/) so it's accessible without sign-in.

In [ ]:
import pandas as pd

df = pd.read_csv('http://hosting.mlmath.wedeploy.io/kc_house_data.csv')

To summarize, our goal now is to create a model that will predict the prices of houses. From the machine learning perspective, we'd like to simply present the data to the model, and the model will figure out the appropriate way to guess at the price of our houses.

## Define Incorrectness

So, let's take a quick step back. You will be asking the model to train itself, and it will provide increasingly correct answers as outputs. How does the model know what to do in order to know how to get increasing correctness?

Essentially, it knows because you have a definition of "degree of incorrectness". In machine learning, this is referred to as a *cost function*.

Intuitively, the cost function provides a way for the model to detect whether it got better or if it got worse.

There are a variety of considerations that you take into account when saying one model is better than another, but one of the more obvious elements of the cost function is the difference between what the model guessed $\hat{y_i}$ as compared to the true correct answer $y_i$.

The notion of a guess can vary between machine learning models. Some guess at a number in the range $(-\infty, +\infty)$. Some guess in a more bounded range, such as $[0, 1]$ or $[-1, 1]$. Still others might have multiple outputs. In all cases, you define your sense of right and wrong for any given example, and then you apply a function (sum and average are common) and declare that to be the degree of incorrectness of the model.

The simplest definition of right and wrong is binary. In this world, every incorrect answer is weighted equally. Either you got it exactly right or you got it wrong. As a cost function, this is known as **zero-one loss**.

$$
L(y_i, \hat{y_i}) =
\begin{cases}
0 & y = \hat{y_i} \\
1 & y \ne \hat{y_i}
\end{cases}
$$

Another simple definition is to take the absolute value of the difference between $y_i$ and $\hat{y_i}$. In this world, every incorrect answer is weighted according to how far away it was from the true value. As a cost function, this is known as **absolute loss**.

$$
L(y_i, \hat{y_i}) = | y_i - \hat{y_i} |
$$

One of the more common loss functions is to take the square of the difference between $y_i$ and $\hat{y_i}$. In this world, the farther you are away from the true value, the more dramatic the penalty for that incorrectness. As a cost function, this is known as **squared loss**.

$$
L(y_i, \hat{y_i}) = ( y_i - \hat{y_i} )^2
$$

### Checkpoint: Define Incorrectness

In our case, we are predicting the prices of houses. We don't expect to have this guess be perfect, but we'd like to penalize guesses that are way off.

Because we wish to penalize guesses differently based on how far off they are, we rule out zero-one loss. Because we would like our loss function to penalize really wrong guesses more heavily, **squared loss** makes more sense for us than absolute loss.

After summing the squared losses, we'll wind up with the sum of squared errors, or SSE.

$$
\text{SSE} = \sum\limits_{i=1}^m(y_i - \hat{y_i})^2
$$

However, this number is huge, and we aren't able to relate it back to what it is that we're predicting, so we don't have a good sense of just how wrong we are, just that smaller numbers are preferred. How can we fix that?

The first transformation is to compute the average of these squared differences rather than just their sum. This will give us the average of the squared errors, which reins in the values so that even if you add more data points, the value does not increase if the errors are about the same across the board. This transformation is known as the mean-square error, or MSE.

$$
\text{MSE} = \frac{\text{SSE}}{m} = \frac{1}{m} \sum\limits_{i=1}^m(y_i - \hat{y_i})^2
$$

If we have the average squared error, then if we take the square root, we have something that we can relate to our output value in a more obvious way. This transformation is known as root-mean-square error, or RMSE.

$$
\text{RMSE} = \sqrt{\text{MSE}} = \sqrt{\frac{1}{m} \sum\limits_{i=1}^m(y_i - \hat{y_i})^2}
$$

This is the number we will use to define the incorrectness of our model.

## Baseline Model

Once you've defined a cost function, all machine learning follows with the following question. What is the simplest almost-oracle I can think of related to this degree of incorrectness?

* [How to Get Baseline Results and Why They Matter](http://machinelearningmastery.com/how-to-get-baseline-results-and-why-they-matter/)

Answering this question gives you a meaningful starting point and it allows you to say whether your self-adjusting almost-oracle is actually any better than the simplest almost-oracle.

The simplest almost-oracle in many cases is simply always guessing the exact same value, no matter what the input values are. For that reason, you will often choose an almost-oracle that is equivalent to the central tendency measure connected to your definition of incorrectness.

* [Modes, Medians, Means: A Unifying Perspective](http://www.johnmyleswhite.com/notebook/2013/03/22/modes-medians-and-means-an-unifying-perspective/)

To summarize the article above:

* For zero-one loss, you would choose the mode of the output values as the baseline model.
* For absolute loss, you would choose the median of the output values.
* For squared loss, which encompasses the [sum of squared errors](https://en.wikipedia.org/wiki/Residual_sum_of_squares), the [mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error), or the [root-mean-square error](https://en.wikipedia.org/wiki/Root-mean-square_deviation), you would choose the mean of the output values.

### Checkpoint: Create the Baseline Model

First, let's create our baseline model by computing the mean of all prices.

In [ ]:
baselineModel = df['price'].mean()

baselineModel

To put what this baseline model means in plain English, we have now created a model that will simply always guess that the price of the house is \$540,088 USD, no matter what.

Now that we have our baseline model, we'll want to compute the incorrectness of this baseline model. Since we chose RMSE as our degree of incorrectness measure, all we have to do is compute it. The first step is to find out how off we are, which we can compute simply by subtracting the true price $y_i$ and our model's guess $\hat{y_i}$.

In [ ]:
baselineModelError = df['price'] - baselineModel

If we square each of these errors and compute the mean of the squared values, we will have the mean squared error, MSE. If we compute the square root of that, then we have the root mean squared error, RMSE.

In [ ]:
import numpy as np

baselineModelSquaredError = baselineModelError ** 2
baselineModelRMSE = np.sqrt(baselineModelSquaredError.mean())
baselineModelRMSE

## Linear Model Overview

When you choose linear regression as a model, you essentially say that the output variable is a linear combination of the input variables. More explicitly, we have a vector of *weights* that contains $n$ entries (for each of the features) and an additional intercept term $\beta_0$:

$$
\begin{bmatrix}
\beta_0 \\
\vdots \\
\beta_n
\end{bmatrix}
$$

If we were to take the numerical value we gave to each feature $x_{i,j}$ and multiply it by the corresponding $\beta_j$ and then sum these values along with the intercept term $\beta_0$, we have an estimate of the result variable $\hat{y_i}$.

$$
\hat{y_i} =
\begin{bmatrix}
1 & x_{i,1} & \cdots & x_{i,n}
\end{bmatrix}
\times
\begin{bmatrix}
\beta_0 \\
\vdots \\
\beta_n
\end{bmatrix}
$$

Choosing a linear regression model is equivalent to saying that we can estimate the output vector by transforming the input matrix with a single matrix multiplication.

$$
\begin{bmatrix}
\hat{y_1} \\
\hat{y_2} \\
\vdots \\
\hat{y_m}
\end{bmatrix}
=
\begin{bmatrix}
1 & x_{1,1} & \cdots & x_{1,n} \\
1 & x_{2,1} & \cdots & x_{2,n} \\
\vdots & \vdots & \ddots & \vdots \\
1 & x_{m,1} & \cdots & x_{m,n}
\end{bmatrix}
\times
\begin{bmatrix}
\beta_0 \\
\vdots \\
\beta_n
\end{bmatrix}
$$

## Linear Model Assumptions

Some of the basic assumptions you make with a linear model are summarized in the following discussion.

* [Linear Models](http://www.stat.berkeley.edu/~aditya/resources/LectureFOUR.pdf)

We'll go into the more advanced understanding of these assumptions in our next lesson, but for now, we will only rely on the simplest assumption, which is that we assume that the input variables $x_{i,j}$ are non-random and that the result variables $y_i$ are random.

If the idea of a "random variable" as a statistical concept is unfamiliar to you, it's good to do a quick refresher of drawing samples from a probability distribution.

* [Chapter 1 of Introduction to Stochastic Processes](https://www.ma.utexas.edu/users/gordanz/notes/introduction_to_stochastic_processes.pdf)

Essentially, we assume that our measurements of each of the $x_{i,j}$ variables are precise values and are not random, but $y_i$ will vary from the model estimate $\hat{y_i}$ by an error term $\epsilon_i$.

In an ideal world where all of the linear model assumptions are satisfied, you can say something special about $\epsilon_i$, which is that while we don't actually expect our guesses to be exactly right, we will be wrong in a way that looks like a zero-mean random variable.

* [White Noise](https://en.wikipedia.org/wiki/White_noise#Mathematical_definitions)

We aren't going to assume ideal world today, so what we will instead say for now is that if we know that our $x_{i,j}$ values are non-random and our $y_i$ values are random, we can say that the true $y_i$ values relate to our model's guesses $\hat{y_i}$ values in the following way, though we know very little about the nature of $e_i$.

$$
\begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_m
\end{bmatrix}
=
\begin{bmatrix}
\hat{y_1} \\
\hat{y_2} \\
\vdots \\
\hat{y_m}
\end{bmatrix}
+
\begin{bmatrix}
\epsilon_1 \\
\epsilon_2 \\
\vdots \\
\epsilon_m
\end{bmatrix}
$$

## Indicator/Boolean Variables

### Checkpoint: Indicator/Boolean Variables

## Linear Regression, Take 1

$$
y_i = \widehat{\beta_0} + \widehat{\beta_1} x_1 + \widehat{\epsilon_i}
$$

## Meaningful Numeric Variables

### Checkpoint: Meaningful Numeric Variables

## Less Meaningful Numeric Variables

### Checkpoint: Less Meaningful Numeric Variables

## Linear Regression, Take 2

$$
y_i = \widehat{\beta_0} + \widehat{\beta_1} x_1 + \dotsb + \widehat{\beta_n} x_n + \widehat{\epsilon_i}
$$

## Transformed Variables

### Checkpoint: Transformed Variables

## Interaction Terms

### Checkpoint: Interaction Terms

## Linear Regression, Take 3

$$
y_i = \widehat{\beta_0} + \widehat{\beta_1} x_1 + \dotsb + \widehat{\beta_n} x_n + \widehat{\epsilon_i}
$$

## Text Vectorization

### Checkpoint: Text Vectorization

## Linear Regression, Take 4

$$
y_i = \widehat{\beta_0} + \widehat{\beta_1} x_1 + \dotsb + \widehat{\beta_n} x_n + \widehat{\epsilon_i}
$$

## Closing Thoughts

Hopefully you have now become curious about linear regression.

You might wonder about simple extensions to linear regression, such as linear spline regression where you have boundary points where the coefficients completely change.

* [An Introduction to Splines](http://www.statpower.net/Content/313/Lecture%20Notes/Splines.pdf)

You might also wonder why we talked about cost functions at the start and if choosing a different cost function might change the way the regression works.

* [Quantile Regression: An Introduction](http://www.econ.uiuc.edu/~roger/research/intro/rq3.pdf)

It's likely that you've also been wondering about applying transformations of the input and output variables in order to overcome the constraints of the linear relationship between variables.

* [Transformations in Regression](http://people.stern.nyu.edu/jsimonof/classes/2301/pdf/transfrm.pdf)

You might be able to follow examples of people looking at input types that we haven't talked about (such as geospatial data) that will allow you to apply linear regression to other problems that involve the prediction of a continuous variable with range $(-\infty, \infty)$.

* [AirBnb Properties in Boston](https://github.com/ResidentMario/boston-airbnb-geo/blob/master/notebooks/boston-airbnb-geo.ipynb)

However, before we let you get into any of that, the next important thing we want you to understand is the math behind the cost function(s) that might be applied when performing linear regression. That will be our next lesson.

* [Usual assumptions for linear regression](http://stats.stackexchange.com/questions/16381/what-is-a-complete-list-of-the-usual-assumptions-for-linear-regression)
* Convexity of cost functions
* Stochastic gradient descent
* Different interpretations of distance (euclidean, etc.)
* Regularization

## Additional Resources